<a href="https://colab.research.google.com/github/blindauth/abstention_experiments/blob/master/hema/AnalyzeHemeResults.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!rm -r abstention
!pip uninstall abstention
!git clone https://github.com/blindauth/abstention.git
!pip install abstention/
![[ -e abstention_experiments ]] || git clone https://github.com/blindauth/abstention_experiments.git

Uninstalling abstention-0.1.2.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/abstention-0.1.2.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/abstention/*
Proceed (y/n)? y
  Successfully uninstalled abstention-0.1.2.1
Cloning into 'abstention'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 51 (delta 18), reused 42 (delta 9), pack-reused 0
Unpacking objects: 100% (51/51), done.
Processing ./abstention
  Stored in directory: /tmp/pip-ephem-wheel-cache-3gjxge4k/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention


In [0]:
import gzip
import json

task0_abstention_results =\
  json.loads(gzip.open(
    "abstention_experiments/hema/heme_task0_abstention_results.json.gz").read())
task1_abstention_results =\
  json.loads(gzip.open(
    "abstention_experiments/hema/heme_task1_abstention_results.json.gz").read())

In [7]:
from abstention.figure_making_utils import (
    wilcox_srs, get_ustats_mat,
    get_tied_top_and_worst_methods)
import numpy as np
from collections import defaultdict

metricnames = ['auroc', 'tpratfpr_0.2']
abstfracs = ['0.15', '0.3']

abstfrac_to_metricname_to_methodnames = {
    '0.15': {
        'auroc': ['distfrom0.5:none',
                  'distfrom0.5:platt',
                  #'js:none',
                  'js:platt',
                  #'estwindowdelta_auroc_15pc:none',
                  'estwindowdelta_auroc_15pc:platt',
                  'fumera_auroc_15pc:none',
                  'fumera_auroc_15pc:platt',
                  'uncertainty:none',
                  'uncertainty:platt'
                 ],
        'tpratfpr_0.2': ['distfrom0.5:none',
                         'distfrom0.5:platt',
                         #'js:none',
                         'js:platt',
                         #'mcwindowdelta_tpratfpr_15pc:none',
                         'mcwindowdelta_tpratfpr_15pc:platt',
                         'fumera_tpratfpr_15pc:none',
                         'fumera_tpratfpr_15pc:platt',
                         'uncertainty:none',
                         'uncertainty:platt']
    },
    '0.3': {
        'auroc': ['distfrom0.5:none',
                  'distfrom0.5:platt',
                  #'js:none',
                  'js:platt',
                  #'estwindowdelta_auroc_30pc:none',
                  'estwindowdelta_auroc_30pc:platt',
                  'fumera_auroc_30pc:none',
                  'fumera_auroc_30pc:platt',
                  'uncertainty:none',
                  'uncertainty:platt'],
        'tpratfpr_0.2': ['distfrom0.5:none',
                         'distfrom0.5:platt',
                         #'js:none',
                         'js:platt',
                         #'mcwindowdelta_tpratfpr_30pc:none',
                         'mcwindowdelta_tpratfpr_30pc:platt',
                         'fumera_tpratfpr_30pc:none',
                         'fumera_tpratfpr_30pc:platt',
                         'uncertainty:none',
                         'uncertainty:platt']
    }   
}

nicemethodname_to_calib_metric_to_abstfrac_to_methodname = {
    "Est $\Delta$Metric (Ours)": {
        'yescalib': {
          'auroc': {
              '0.3': 'estwindowdelta_auroc_30pc:platt',
              '0.15': 'estwindowdelta_auroc_15pc:platt'
          },
          'tpratfpr_0.2': {
              '0.3': 'mcwindowdelta_tpratfpr_30pc:platt',
              '0.15': 'mcwindowdelta_tpratfpr_15pc:platt'
          }
      }
    },
    "JS Div. (Ours)": {
        'yescalib': {
          'auroc': {'0.3': 'js:platt', '0.15': 'js:platt'},
          'tpratfpr_0.2': {'0.3': 'js:platt', '0.15': 'js:platt'}
        }
    },
    "Dist. from 0.5": {
        'yescalib': {
          'auroc': {'0.3': 'distfrom0.5:platt', '0.15': 'distfrom0.5:platt'},
          'tpratfpr_0.2': {'0.3': 'distfrom0.5:platt', '0.15': 'distfrom0.5:platt'}
        }, 'nocalib': {
          'auroc': {'0.3': 'distfrom0.5:none', '0.15': 'distfrom0.5:none'},
          'tpratfpr_0.2': {'0.3': 'distfrom0.5:none', '0.15': 'distfrom0.5:none'}
        }
    },
    "Test-time Dropout": {
        'yescalib': {
          'auroc': {'0.3': 'uncertainty:platt', '0.15': 'uncertainty:platt'},
          'tpratfpr_0.2': {'0.3': 'uncertainty:platt', '0.15': 'uncertainty:platt'}
        }, 'nocalib': {
          'auroc': {'0.3': 'uncertainty:none', '0.15': 'uncertainty:none'},
          'tpratfpr_0.2': {'0.3': 'uncertainty:none', '0.15': 'uncertainty:none'}
        }
    },
    "Fumera et al.": {
        'yescalib': {
          'auroc': {
              '0.3': 'fumera_auroc_30pc:platt',
              '0.15': 'fumera_auroc_30pc:platt'
          },
          'tpratfpr_0.2': {
              '0.3': 'fumera_tpratfpr_30pc:platt',
              '0.15': 'fumera_tpratfpr_15pc:platt'
          }
      }, 'nocalib': {
          'auroc': {
              '0.3': 'fumera_auroc_30pc:none',
              '0.15': 'fumera_auroc_15pc:none'
          },
          'tpratfpr_0.2': {
              '0.3': 'fumera_tpratfpr_30pc:none',
              '0.15': 'fumera_tpratfpr_15pc:none'
          }
      }
    }
}
nicemethodname_orders = ["Est $\Delta$Metric (Ours)", "JS Div. (Ours)",
                         "Dist. from 0.5", "Fumera et al.", "Test-time Dropout"]

def stdderr(vals):
  return np.std(vals, ddof=1)/np.sqrt(len(vals))

for taskidx, abstention_results in [
    ('task0', task0_abstention_results),
    ('task1', task1_abstention_results)]:
  
  #identify the best methods
  metricname_to_abstfrac_to_bestmethods = defaultdict(dict)  
  for metricname in metricnames:
    for abstfrac in abstfracs:
      methodnames = abstfrac_to_metricname_to_methodnames[abstfrac][metricname]
      nsamples = len(list(
        abstention_results['metricname_to_abstfrac_to_abstncalib_to_perf']
                          [metricname][abstfrac].values())[0])
      ustats_mat = get_ustats_mat(
          method_to_perfs=
            abstention_results['metricname_to_abstfrac_to_abstncalib_to_perf']
                [metricname][abstfrac],
          method_names=methodnames,
          max_ustat=nsamples*(nsamples+1)/2 )
      tied_top_methods, tied_worst_methods =(
          get_tied_top_and_worst_methods(
              ustats_mat,
              methodnames,
              #0.05 threshold for one-sided test from normal approx when N=30,
              #from https://www.oreilly.com/library/view/nonparametric-statistics-a/9781118840429/bapp02.xhtml
              threshold=151
          ))
      tied_top_methods = [methodnames[x] for x in tied_top_methods]
      print("bestmethods",metricname, abstfrac, tied_top_methods)
      metricname_to_abstfrac_to_bestmethods[metricname][abstfrac] =\
        set(tied_top_methods)
      
  #generate the latex table
  thetable = """
\\begin{table*}[!h]
\\tiny
\\begin{center}
\\begin{tabular}{ | c c | c c c | c c c | }
\\hline
\\multicolumn{2}{| c |}{ } & \\multicolumn{3}{ c |}{Sensitivity @ 80\\% Specificity} & \\multicolumn{3}{c | }{Area under ROC}\\\\
\\multicolumn{1}{| c}{Method} & \\multicolumn{1}{c |}{Calib?} & Base & @30\\% Abst. & @15\\% Abst. & Base & @30\\% Abst. & @15\\% Abst.\\\\ \\hline
"""
 
  metricname_to_baselineperfs =\
    abstention_results['metricname_to_baselineperfs']
  metricname_to_abstfrac_to_abstncalib_to_perf =\
    abstention_results['metricname_to_abstfrac_to_abstncalib_to_perf']

  for nicemethodname in nicemethodname_orders:
    for calib in ['yescalib', 'nocalib']:   
      if calib in nicemethodname_to_calib_metric_to_abstfrac_to_methodname[nicemethodname]:
        thetable += nicemethodname
        thetable += " & "+("Y" if calib=='yescalib' else 'N')
        for metricname in ['tpratfpr_0.2', 'auroc']:          
          baseperfs = metricname_to_baselineperfs[metricname]
          thetable += (" & "+str(np.round(np.mean(baseperfs),4))
                       +"$\\pm$"+str(np.round(stdderr(baseperfs),4)))
          for abstfrac in ['0.3', '0.15']:
            methodname =\
             nicemethodname_to_calib_metric_to_abstfrac_to_methodname[
              nicemethodname][calib][metricname][abstfrac]
            isbest = (methodname in
              metricname_to_abstfrac_to_bestmethods[metricname][abstfrac])
            methodperfs = metricname_to_abstfrac_to_abstncalib_to_perf[
              metricname][abstfrac][methodname]
            thetable += " & "
            if (isbest):
              thetable += "\\textbf{"
            thetable += (str(np.round(np.mean(methodperfs),4))
                         +"$\pm$"+str(np.round(stdderr(methodperfs),4)))
            if (isbest):
              thetable += "}"
        thetable += "\\\\ \\hline\n"
        if (nicemethodname=="JS Div. (Ours)"):
          thetable += "\\hline\n"
  
  thetable += """\\end{tabular}
\\end{center}
\\caption{\\textbf{}. }
\\label{tab:heme_"""+taskidx+"""}
\\end{table*}
"""
  print("\n\n"+thetable)

bestmethods auroc 0.15 ['estwindowdelta_auroc_15pc:platt']
bestmethods auroc 0.3 ['estwindowdelta_auroc_30pc:platt']
bestmethods tpratfpr_0.2 0.15 ['mcwindowdelta_tpratfpr_15pc:platt']
bestmethods tpratfpr_0.2 0.3 ['mcwindowdelta_tpratfpr_30pc:platt']



\begin{table*}[!h]
\tiny
\begin{center}
\begin{tabular}{ | c c | c c c | c c c | }
\hline
\multicolumn{2}{| c |}{ } & \multicolumn{3}{ c |}{Sensitivity @ 80\% Specificity} & \multicolumn{3}{c | }{Area under ROC}\\
\multicolumn{1}{| c}{Method} & \multicolumn{1}{c |}{Calib?} & Base & @30\% Abst. & @15\% Abst. & Base & @30\% Abst. & @15\% Abst.\\ \hline
Est $\Delta$Metric (Ours) & Y & 0.6875$\pm$0.0031 & \textbf{0.8371$\pm$0.0027} & \textbf{0.7638$\pm$0.0027} & 0.8309$\pm$0.0012 & \textbf{0.883$\pm$0.0011} & \textbf{0.8567$\pm$0.0012}\\ \hline
JS Div. (Ours) & Y & 0.6875$\pm$0.0031 & 0.8288$\pm$0.0031 & 0.761$\pm$0.0029 & 0.8309$\pm$0.0012 & 0.8828$\pm$0.0011 & 0.8566$\pm$0.0012\\ \hline
\hline
Dist. from 0.5 & Y & 0.6875$\pm$0.0031 & 0.7

In [5]:
metricname_to_abstfrac_to_abstncalib_to_perf[
              metricname][abstfrac].keys()

dict_keys(['distfrom0.5:none', 'distfrom0.5:platt', 'estwindowdelta_auroc_15pc:none', 'estwindowdelta_auroc_15pc:platt', 'estwindowdelta_auroc_30pc:none', 'estwindowdelta_auroc_30pc:platt', 'fumera_auroc_15pc:none', 'fumera_auroc_15pc:platt', 'fumera_auroc_30pc:none', 'fumera_auroc_30pc:platt', 'fumera_tpratfpr_15pc:none', 'fumera_tpratfpr_15pc:platt', 'fumera_tpratfpr_30pc:none', 'fumera_tpratfpr_30pc:platt', 'js:none', 'js:platt', 'mcwindowdelta_tpratfpr_15pc:none', 'mcwindowdelta_tpratfpr_15pc:platt', 'mcwindowdelta_tpratfpr_30pc:none', 'mcwindowdelta_tpratfpr_30pc:platt', 'uncertainty:none', 'uncertainty:platt'])